In [ ]:

fasta_path = '../data/sample_proteins.fasta'

# Read the file contents into a single string named `raw_fasta_text`.
with open(fasta_path, 'r') as fasta_file:
    raw_fasta_text = fasta_file.read() #als string auslesen

: 

In [ ]:


def read_fasta(fasta_path):
    """
    Function converts text into a dictionary with seq and ids.

    Parameters
    ----------
     fasta_path : str or path-like
        Path to a FASTA file containing protein sequences. Header lines are
        expected to start with '>' and contain UniProt-style IDs separated
        by '|' (e.g. '>sp|P12345|Description').
    Returns
    -------
    dict of str to str
        Dictionary mapping protein IDs to their full amino-acid sequences
        (one continuous string per protein).
    """


    protein_map = {} # create an empty dictionary 
    current_id = None 
    current_sequence = [] #create an empty list 

    with open(filepath, 'r', encoding='utf-8') as fasta_handle: # opens the file and delets whitespaces
        for line in fasta_handle:
            stripped = line.strip()  
            if not stripped:
                continue
            if stripped.startswith('>'): # header line in FASTA
                if current_id is not None: #so we skip the first line as the first id is = 0 => None
                    protein_map[current_id] = ''.join(current_sequence) # 
                    current_sequence = []
                current_id = stripped.split('|')[1] # takes the second charachter and delets the bars
            else:
                current_sequence.append(stripped) #chooses the stripped part 

    if current_id is not None: # saves the previous protein
        protein_map[current_id] = ''.join(current_sequence)

    return protein_map

In [ ]:
import re
enzyme_cleavage_patterns = {
        'LysC': r'(?<=K)',
        'LysN': r'(?=K)',
        'ArgC': r'(?<=R)',
        'Trypsin': r'(?<=[KR])(?!P)', 
    }




def digest_protein_sequence(protein_seq, cleave_pattern): # defines the new function
    """
    Function description
    Digest a protein sequence into peptides using a regex cleavage pattern.
    Parameters
    ----------
    protein_seq : str
        Amino-acid sequence of the protein (one-letter code).
    cleave_pattern : str
        Regular expression describing the cleavage sites, e.g.
        enzyme_cleavage_patterns['LysC'].
    Returns
    -------
    list of str
        List of peptide sequences produced by the in-silico digestion.
    """
    peptides = re.split(protein_seq, cleave_pattern)
    
    enzyme_cleavage_patterns = {
        'LysC': r'(?<=K)',
        'LysN': r'(?=K)',
        'ArgC': r'(?<=R)',
        'Trypsin': r'(?<=[KR])(?!P)', 
    }
    #sequence = protein_seq # aqui he cambiado algo

    cleave_pattern = enzyme_cleavage_patterns[cleave_pattern]


    print(peptides)
    print(f'Nr. of digested peptides: {len(peptides)}')



def digest_protein_collection(protein_map, enzyme_name, min_pep_len=5, max_pep_len=30):
    """
    Digest a collection of protein sequences into peptides using a regex cleavage pattern. 

    Parameters
    ----------
    protein_map : dict
        Dictionary mapping protein IDs to amino-acid sequences (one-letter code),
        e.g. {"P11802": "MKK....", ...}
    enzyme_name : str
        Name of the enzyme, one of: 'LysC', 'LysN', 'ArgC', 'Trypsin'.
    min_pep_len : int
        Minimum allowed peptide length (in amino acids).
    max_pep_len : int
        Maximum allowed peptide length (in amino acids).

    Returns
    -------
    dict
        Dictionary mapping protein IDs to a list of digested peptides that pass
        the length filters.
    """

    enzyme_cleavage_patterns = {
        'LysC':   r'(?<=K)',
        'LysN':   r'(?=K)',
        'ArgC':   r'(?<=R)',
        'Trypsin': r'(?<=[KR])(?!P)',   # cut after K/R, but not if followed by P
    }

    # Look up the regex pattern for the chosen enzyme
    pattern = enzyme_cleavage_patterns[enzyme_name]

    # This will store the final result
    digested_dict = {}

    # Loop over all proteins in the collection
    for protein_id, sequence in protein_map.items():
        # Split the sequence at the cleavage pattern
        peptides = re.split(pattern, sequence)

        # Filter peptides by length (keep only those in the desired range)
        filtered_peptides = [
            pep for pep in peptides
            if min_pep_len <= len(pep) <= max_pep_len
        ]

        if not filtered_peptides:
            print(f"{protein_id}: does not fit criteria")
            # go on to the next protein instead of stopping the whole function
            continue

        print(f"{protein_id}: {filtered_peptides}")
        print(f'Nr. of digested peptides: {len(filtered_peptides)}')

        # Save peptides for this protein in the output dictionary
        digested_dict[protein_id] = filtered_peptides

    # Only return once, after the loop is done
    return digested_dict



def compute_sequence_coverage(protein_seq, peptides):
    """
    Add a short description here.

    Parameters
    ----------
    protein_seq = based on the orginal sequence from FASTA
    peptides = digested using function 
    Returns
    -------
    """
    if not protein_seq:
        return 0.0

    covered = [False] * len(protein_seq)

    for pep in peptides:
        if not pep:
            continue

        # find first occurrence of the peptide in the protein
        idx = protein_seq.find(pep)
        if idx == -1:
            continue

        # mark covered positions
        for i in range(idx, idx + len(pep)):
            if i < len(protein_seq):
                covered[i] = True

    return sum(covered) / len(protein_seq) * 100.0
    

In [ ]:
from pyteomics import achrom

def predict_lc_retention_times(peptides): 
    """
    peptides: list of peptide sequences, e.g. ["MATSR", "PEPTIDE", ...]
    Returns: dict {peptide_sequence: retention_time}
    """
    relative_RT = {
        p: achrom.calculate_RT(p, achrom.RCs_guo_ph7_0)
        for p in peptides  # p is e.g. "MATSR"
    }
    print(relative_RT)
    return relative_RT

import matplotlib.pyplot as plt

def plot_retention_time(retention_times, resolution=30):
    """
    Add a short description here.

    Parameters
    ----------

    Returns
    -------

    """
    if retention_times == dict : 
        retention_times = list()
    
    else : 

        plt.hist(retention_times, resolution)   # you can change bins
        plt.xlabel("")
        plt.ylabel("a.u(retention time)")
        plt.title("Histogram of retention times")
        plt.show()


def select_retention_time_window(peptide_rt_map, lower_ret_time, upper_ret_time):
    """
    peptide_rt_map : dict
        {peptide_sequence: retention_time}
    lower_ret_time, upper_ret_time : float
        Allowed RT range.
    """
    filtered = {}  # will store only peptides within the RT range

    for pep, rt in peptide_rt_map.items():
        if lower_ret_time <= rt <= upper_ret_time:
            filtered[pep] = rt
        
    return filtered

In [ ]:
amino_acid_mass_dalton = {
    'A': 71.08, 'R': 156.19, 'N': 114.10, 'D': 115.09,
    'C': 103.15, 'E': 129.12, 'Q': 128.13, 'G': 57.05,
    'H': 137.14, 'I': 113.16, 'L': 113.16, 'K': 128.17,
    'M': 131.19, 'F': 147.18, 'P': 97.12, 'S': 87.08,
    'T': 101.11, 'W': 186.21, 'Y': 163.18, 'V': 99.13,
}



def calculate_mol_mass(peptide_seq, amino_acid_mass_dict=None):
    """
    Calculate the molecular mass of a single peptide sequence and
    return a dictionary {peptide_seq: mass}.

    Parameters
    ----------
    peptide_seq : str
        Peptide sequence (one-letter amino-acid codes).
    amino_acid_mass_dict : dict, optional
        Mapping from amino-acid letter to mass in Dalton. If None,
        amino_acid_mass_dalton is used.

    Returns
    -------
    dict
        {peptide_seq: total_mass_in_dalton}
    """
    if amino_acid_mass_dict is None:
        amino_acid_mass_dict = amino_acid_mass_dalton

    total_mass = 0.0
    for aa in peptide_seq:
        total_mass += amino_acid_mass_dict[aa]

    return {peptide_seq: total_mass}


def calculate_mol_mass_collection(peptides, amino_acid_mass_dict=None):
    """
    Calculate the molecular mass for a list of peptide sequences.

    Parameters
    ----------
    peptide_seqs : list of str
        Peptide sequences, e.g. ["MATSR", "PEPTIDE", ...]
    amino_acid_mass_dict : dict, optional
        Mapping {amino_acid_letter: mass_in_dalton}.
        If None, amino_acid_mass_dalton is used.

    Returns
    -------
    dict
        {peptide_sequence: total_mass_in_dalton}
    """
    if amino_acid_mass_dict is None:
        amino_acid_mass_dict = amino_acid_mass_dalton

    result = {}

    for peptide in peptides:
        total_mass = 0.0
        for aa in peptide:
    
            total_mass += amino_acid_mass_dict[aa]

        result[peptide] = total_mass

    return result


def calculate_mz_collection(peptide_mass_map, charge=2, proton_mass=1.007):
    """
    Convert a peptide→mass dictionary to a peptide→m/z dictionary.

    Parameters
    ----------
    peptide_mass_dict : dict
        Mapping {peptide_sequence: neutral_mass_in_dalton}.
    charge : int, optional
        Charge state (z). Default is 2.
    proton_mass : float, optional
        Mass of a proton in Dalton. Default is 1.007.

    Returns
    -------
    dict
        {peptide_sequence: result}
    """
    mz_dict = {}

    for peptide, mass in peptide_mass_map.items():
        mz = (mass + charge * proton_mass) / charge
        mz_dict[peptide] = mz

    return mz_dict



import numpy as np
import matplotlib.pyplot as plt

def plot_ms1(mz_dict, random_count_range=(10, 100), seed=42):
    """
    Plot a simple mass spectrum from m/z values with random intensities
    drawn using NumPy.

    Parameters
    ----------
    mz_dict : dict
        {peptide_sequence: m_over_z_value}
    random_count_range : tuple (min_intensity, max_intensity)
    seed : int or None
        Random seed for reproducible intensities. If None, no seed is set.
    """
    if not isinstance(mz_dict, dict):
        raise TypeError(f"mz_dict must be a dict, got {type(mz_dict)}")

    # Optionally set the seed for reproducibility
    rng = np.random.default_rng(seed)

    # m/z values as a NumPy array (used on the x-axis)
    mz_values = np.array(list(mz_dict.values()), dtype=float)

    # Create random intensities
    min_count, max_count = random_count_range
    intensities = rng.integers(
        low=min_count,
        high=max_count + 1,   # upper bound is exclusive
        size=len(mz_values)
    )

    # Plot
    
    plt.bar(mz_values, intensities)
    plt.xlabel("m/z")
    plt.ylabel("Intensity (a.u.)")
    plt.title("Simulated MS1 Spectrum")
    
    plt.show()



def fragment_peptide(peptide):
    """
    Generate b- and y-ion fragment sequences for a peptide.

    Parameters
    ----------
    peptide_seq : str
        Peptide sequence, e.g. "PEPTIDE".

    Returns
    -------
    list of str
        Combined list of b- and y-ion sequences.
        First all b-ions (N-terminal), then all y-ions (C-terminal).
    """
    b_ions = []  # from N-terminus: P, PE, PEP, ...
    y_ions = []  # from C-terminus: E, DE, IDE, ...


    # i goes from 1 up to n (inclusive)
    for i in range(1, len(peptide)):
        # b-ion: take the first i amino acids
        b_ions.append(peptide[:i])

        # y-ion: take the last i amino acids
        y_ions.append(peptide[len(peptide) - i:]) # start to end

    # return one combined list: all b-ions followed by all y-ions
    return b_ions + y_ions

fragment_peptide("PEPT")